In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [6]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [7]:

loss_thresh = 10000
x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))


lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [8]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [9]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [10]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)         
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [11]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [12]:
def data_update(loss_np):
    train_loss.append(loss_np)
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [13]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [14]:
nan_tune = []
for tune_reps in range(5):
    label = "3D_HTTP_tanh_tune"+str(tune_reps)
    max_reps = 10
    max_iter = 100

    train_loss_full = []
    test_mse_full = []
    test_re_full = []

    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))

    
    for reps in range(max_reps):
        print(label)
        train_loss = []
        test_mse_loss = []
        test_re_loss = []

        print(reps)

        torch.manual_seed(reps*36)
        N_D = 5000 #Total number of data points for 'y'
        N_N = 3500
        N_f = 10000 #Total number of collocation points 

        layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
 
        PINN = Sequentialmodel(layers)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())


        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')



        nan_flag = train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time

        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label}
    savemat(label+'.mat', mdic)

3D_HTTP_tanh_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1240582.0 Test MSE 311009.60979152017 Test RE 0.988612884740094
1 Train Loss 846108.0 Test MSE 302579.47305305104 Test RE 0.9751223110224558
2 Train Loss 661049.0 Test MSE 292627.0160867751 Test RE 0.9589513427938866
3 Train Loss 556578.4 Test MSE 292195.68191692117

96 Train Loss 207705.08 Test MSE 64843.84018699844 Test RE 0.45141265039041856
97 Train Loss 207687.4 Test MSE 64807.68879776108 Test RE 0.4512867982696502
98 Train Loss 207674.73 Test MSE 64860.03751972885 Test RE 0.4514690260179039
99 Train Loss 207672.03 Test MSE 64871.56968402733 Test RE 0.45150916000315294
Training time: 109.73
3D_HTTP_tanh_tune0
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Li

92 Train Loss 165783.78 Test MSE 42055.14937598779 Test RE 0.3635372788806918
93 Train Loss 165751.12 Test MSE 42072.45800178339 Test RE 0.36361208165177056
94 Train Loss 165723.44 Test MSE 42019.65887510823 Test RE 0.36338385125656475
95 Train Loss 165709.66 Test MSE 41964.48068736326 Test RE 0.36314518381272237
96 Train Loss 165671.69 Test MSE 41876.86607327522 Test RE 0.36276589341058807
97 Train Loss 165617.97 Test MSE 41824.655965715465 Test RE 0.36253968315223783
98 Train Loss 165539.89 Test MSE 41797.435603319165 Test RE 0.36242168974017447
99 Train Loss 165454.94 Test MSE 41864.895850439 Test RE 0.36271404259958223
Training time: 110.99
3D_HTTP_tanh_tune0
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias

88 Train Loss 167591.52 Test MSE 40943.34880054076 Test RE 0.35869972423874286
89 Train Loss 167049.44 Test MSE 40864.46817909891 Test RE 0.35835402586510157
90 Train Loss 166730.22 Test MSE 40630.01880341424 Test RE 0.35732456514792055
91 Train Loss 166393.73 Test MSE 40403.83991071226 Test RE 0.3563286012392161
92 Train Loss 166072.88 Test MSE 40612.91395460867 Test RE 0.35724934212011544
93 Train Loss 165635.27 Test MSE 40638.02647470631 Test RE 0.35735977552656345
94 Train Loss 164856.0 Test MSE 39914.41905319983 Test RE 0.3541638813329993
95 Train Loss 164059.6 Test MSE 39517.21333495137 Test RE 0.3523972559340117
96 Train Loss 163510.72 Test MSE 39245.91979477574 Test RE 0.3511855339998317
97 Train Loss 163070.72 Test MSE 39073.68080118162 Test RE 0.3504140607460301
98 Train Loss 162403.75 Test MSE 39146.52581388217 Test RE 0.3507405468906141
99 Train Loss 162154.5 Test MSE 39117.807496543996 Test RE 0.3506118697455284
Training time: 106.58
3D_HTTP_tanh_tune0
3
Sequentialmodel(
 

84 Train Loss 180923.4 Test MSE 48209.38031691001 Test RE 0.38922895186376916
85 Train Loss 180614.0 Test MSE 48150.590011829765 Test RE 0.3889915512870491
86 Train Loss 180405.72 Test MSE 48067.741908105374 Test RE 0.3886567569793523
87 Train Loss 180170.97 Test MSE 47934.44062892538 Test RE 0.3881174721134022
88 Train Loss 180024.84 Test MSE 47802.60268658899 Test RE 0.387583369284755
89 Train Loss 179834.22 Test MSE 47549.86676259089 Test RE 0.3865574203369419
90 Train Loss 179712.17 Test MSE 47456.25558464885 Test RE 0.38617672607508013
91 Train Loss 179548.17 Test MSE 47464.40300579433 Test RE 0.3862098745963179
92 Train Loss 179338.78 Test MSE 47421.37203322472 Test RE 0.3860347670015043
93 Train Loss 179125.17 Test MSE 47283.440478081335 Test RE 0.385472940672224
94 Train Loss 178859.73 Test MSE 47154.73421516863 Test RE 0.38494795149138555
95 Train Loss 178605.11 Test MSE 47160.24418946647 Test RE 0.38497044118830576
96 Train Loss 178460.52 Test MSE 47139.31643374721 Test RE 0.

...

0 Train Loss 1044594.75 Test MSE 305850.7264183426 Test RE 0.9803792717792523
1 Train Loss 683708.5 Test MSE 294071.2299855828 Test RE 0.9613148058526195
2 Train Loss 530063.5 Test MSE 284797.9599455456 Test RE 0.9460362964262942
3 Train Loss 511685.47 Test MSE 278307.72361102677 Test RE 0.9351946027510409
4 Train Loss 490698.25 Test MSE 262774.6847438026 Test RE 0.9087221709373626
5 Train Loss 476484.03 Test MSE 253256.18639663022 Test RE 0.8921120255336896
6 Train Loss 453854.84 Test MSE 236227.32225032186 Test RE 0.8615974912269202
7 Train Loss 435449.94 Test MSE 220674.38492017984 Test RE 0.832751310063345
8 Train Loss 417482.62 Test MSE 207332.59899158115 Test RE 0.8071851399913881
9 Train Loss 394547.22 Test MSE 181948.1370521119 Test RE 0.7561590752475473
10 Train Loss 360597.75 Test MSE 161808.93139495925 Test RE 0.713083860629975
11 Train Loss 281574.3 Test MSE 89346.30073251006 Test RE 0.5298801494241854
12 Train Loss 243373.34 Test MSE 87226.53255835932 Test RE 0.52355663354

0 Train Loss 1226050.0 Test MSE 314454.2432369199 Test RE 0.9940725738947608
1 Train Loss 722982.75 Test MSE 288631.7811540115 Test RE 0.952382566345589
2 Train Loss 529804.7 Test MSE 277332.41652328544 Test RE 0.9335545073686727
3 Train Loss 463275.7 Test MSE 230285.38828162718 Test RE 0.8506924012662566
4 Train Loss 408564.7 Test MSE 191238.0969125749 Test RE 0.7752228563778247
5 Train Loss 351629.53 Test MSE 146827.3518190085 Test RE 0.6792706425064128
6 Train Loss 270199.72 Test MSE 97452.92636750835 Test RE 0.5533970053075697
7 Train Loss 246158.53 Test MSE 85657.71701869894 Test RE 0.518827048900543
8 Train Loss 241005.08 Test MSE 81760.22686883179 Test RE 0.5068861281545837
9 Train Loss 234997.66 Test MSE 77550.29519667028 Test RE 0.49366358170692215
10 Train Loss 231195.66 Test MSE 75187.66532579485 Test RE 0.4860854953178913
11 Train Loss 227573.06 Test MSE 73633.04639265568 Test RE 0.48103396978405133
12 Train Loss 225128.5 Test MSE 72038.30351930474 Test RE 0.475796343968376

0 Train Loss 1100924.8 Test MSE 309279.8848776719 Test RE 0.9858598948640411
1 Train Loss 638041.94 Test MSE 307940.82736847486 Test RE 0.9837233913892885
2 Train Loss 583089.2 Test MSE 309759.63178534224 Test RE 0.9866242186838606
3 Train Loss 543850.25 Test MSE 294594.94429554406 Test RE 0.9621704325113887
4 Train Loss 505551.47 Test MSE 254785.78766650768 Test RE 0.8948020318175886
5 Train Loss 480547.34 Test MSE 240329.78135887903 Test RE 0.8690467862523849
6 Train Loss 449338.3 Test MSE 213872.3319555304 Test RE 0.8198165183463612
7 Train Loss 422524.78 Test MSE 196042.02647225367 Test RE 0.7848993216974173
8 Train Loss 371371.47 Test MSE 147602.20561423196 Test RE 0.6810606456835521
9 Train Loss 264365.9 Test MSE 86054.17510453486 Test RE 0.5200263320913896
10 Train Loss 248310.06 Test MSE 87236.10166116386 Test RE 0.5235853508988455
11 Train Loss 247201.89 Test MSE 85919.81686073478 Test RE 0.5196202093306239
12 Train Loss 246107.95 Test MSE 85348.16514417484 Test RE 0.517888725

0 Train Loss 1213968.2 Test MSE 314363.58603438374 Test RE 0.9939292679338261
1 Train Loss 630444.25 Test MSE 306679.4003421664 Test RE 0.9817064963395473
2 Train Loss 538073.94 Test MSE 296942.7309941728 Test RE 0.9659968526379242
3 Train Loss 495952.97 Test MSE 259822.8588936537 Test RE 0.9036037836271641
4 Train Loss 453822.97 Test MSE 220752.19734244223 Test RE 0.8328981161492361
5 Train Loss 395457.38 Test MSE 180942.7744962828 Test RE 0.7540670860319703
6 Train Loss 364390.47 Test MSE 159568.65526669758 Test RE 0.708130262611115
7 Train Loss 314753.2 Test MSE 128996.2472897347 Test RE 0.6366898116011178
8 Train Loss 243828.48 Test MSE 86714.90997343241 Test RE 0.5220189286139798
9 Train Loss 240669.89 Test MSE 84529.79224591708 Test RE 0.5153998205399049
10 Train Loss 238192.05 Test MSE 82532.3926553091 Test RE 0.5092740883934816
11 Train Loss 234651.22 Test MSE 79431.94660576942 Test RE 0.4996167213648462
12 Train Loss 231513.75 Test MSE 77536.52906981019 Test RE 0.4936197640974

0 Train Loss 1044575.9 Test MSE 310928.59316127765 Test RE 0.9884841117074729
1 Train Loss 648851.1 Test MSE 306440.8423724118 Test RE 0.9813246000180669
2 Train Loss 548923.1 Test MSE 296932.96604297066 Test RE 0.9659809691211811
3 Train Loss 513665.97 Test MSE 280345.6619118641 Test RE 0.938612389020955
4 Train Loss 487330.6 Test MSE 266527.20348744886 Test RE 0.9151876132563511
5 Train Loss 470904.72 Test MSE 253452.87390820563 Test RE 0.8924583808223318
6 Train Loss 442561.34 Test MSE 229654.2988864306 Test RE 0.8495259545463539
7 Train Loss 432213.84 Test MSE 226974.3659584067 Test RE 0.8445546705523974
8 Train Loss 413558.16 Test MSE 202080.7038079073 Test RE 0.7968962535705817
9 Train Loss 390521.9 Test MSE 190365.75267210748 Test RE 0.7734527222612159
10 Train Loss 378248.88 Test MSE 182190.75814466728 Test RE 0.7566630623324747
11 Train Loss 358248.28 Test MSE 153811.77597432942 Test RE 0.6952390469711445
12 Train Loss 320744.6 Test MSE 127472.02432008948 Test RE 0.63291706217

In [15]:
import scipy.io as sio

In [16]:
for tune_reps in range(5):
    label = "3D_HTTP_tanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.3879731744082139
1   0.34584468408748653
2   0.3660971255207167
3   0.363773018761285
4   0.336893187134138
